In [3]:
import psycopg2

In [23]:
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

sql = '''
SELECT 
    route_id, 
    stop_id_start, 
    ROUND(EXTRACT(EPOCH FROM (MAX(end_stop_time) - MIN(init_stop_time))) / 60.0, 3) AS diff_minutes 
FROM 
    warehouse 
GROUP BY CUBE (route_id, stop_id_start)
'''

cursor_psql.execute(sql)
results = cursor_psql.fetchall()
for row in results:
    print(row)

(None, None, Decimal('399.417'))
('11M', 'GNR2', Decimal('5.250'))
('502', 'JN3', Decimal('6.483'))
('201', 'ACRD2', Decimal('7.767'))
('400', 'TRD6', Decimal('29.333'))
('204', 'AUL2', Decimal('2.100'))
('207', 'ALX2', Decimal('10.067'))
('600', 'PRR5', Decimal('-6.283'))
('11M', 'RBR2', Decimal('7.033'))
('301', 'DC2', Decimal('2.533'))
('305', 'ENX1', Decimal('15.817'))
('905', 'LPDZ', Decimal('9.333'))
('207', 'HSA5', Decimal('9.317'))
('202', 'AAL4', Decimal('-3.717'))
('300', 'HSJ6', Decimal('-19.767'))
('701', 'PLIM2', Decimal('6.317'))
('11M', 'GCRT2', Decimal('0.483'))
('701', 'FTNH2', Decimal('1.833'))
('300', 'MCBL', Decimal('16.633'))
('201', 'BS2', Decimal('9.000'))
('301', 'ISEP2', Decimal('14.900'))
('601', 'BCM3', Decimal('-6.217'))
('201', 'TRD6', Decimal('16.850'))
('502', 'FIG', Decimal('13.333'))
('1M', 'MS2', Decimal('17.267'))
('207', 'BPTL2', Decimal('16.383'))
('202', 'BCM3', Decimal('20.000'))
('202', 'TRD1', Decimal('24.333'))
('401', 'AANT2', Decimal('7.850')

In [24]:
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

sql = '''
SELECT 
    route_id, 
    stop_id_start, 
    COUNT(*)
FROM 
    warehouse 
GROUP BY CUBE (route_id, stop_id_start)
'''

cursor_psql.execute(sql)
results = cursor_psql.fetchall()
for row in results:
    print(row)

(None, None, 100)
('11M', 'GNR2', 1)
('502', 'JN3', 1)
('201', 'ACRD2', 1)
('400', 'TRD6', 1)
('204', 'AUL2', 1)
('207', 'ALX2', 2)
('600', 'PRR5', 1)
('11M', 'RBR2', 2)
('301', 'DC2', 1)
('305', 'ENX1', 1)
('905', 'LPDZ', 1)
('207', 'HSA5', 1)
('202', 'AAL4', 1)
('300', 'HSJ6', 1)
('701', 'PLIM2', 1)
('11M', 'GCRT2', 1)
('701', 'FTNH2', 1)
('300', 'MCBL', 1)
('201', 'BS2', 4)
('301', 'ISEP2', 1)
('601', 'BCM3', 1)
('201', 'TRD6', 1)
('502', 'FIG', 1)
('1M', 'MS2', 3)
('207', 'BPTL2', 1)
('202', 'BCM3', 1)
('202', 'TRD1', 2)
('401', 'AANT2', 2)
('204', 'FG3', 1)
('202', 'BVBR1', 1)
('201', 'GGF', 2)
('502', 'ACRD2', 2)
('304', 'BRA4', 1)
('504', 'LRD1', 1)
('303', 'DJOA4', 1)
('11M', 'MS1', 1)
('207', 'PVC2', 1)
('202', 'BCM4', 2)
('303', 'RA2', 2)
('11M', 'HSJ6', 3)
('703', 'TRD1', 1)
('202', 'GCRT1', 1)
('403', 'RBR2', 1)
('703', 'ARS3', 1)
('703', 'COMB2', 1)
('303', 'LPDZ', 1)
('300', 'DC4', 3)
('502', 'PRR1', 2)
('502', 'PINM3', 1)
('302', 'MRR1', 1)
('201', 'CMO', 1)
('5M', 'MPL4